In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use('fast')

In [2]:
#read test data
test_data = pd.read_csv('./datasets/test.csv')

In [3]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     718 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        878 non-null    object 
 9   Land Contour     878 non-null    object 
 10  Utilities        878 non-null    object 
 11  Lot Config       878 non-null    object 
 12  Land Slope       878 non-null    object 
 13  Neighborhood     878 non-null    object 
 14  Condition 1      878 non-null    object 
 15  Condition 2      878 non-null    object 
 16  Bldg Type        878 non-null    object 
 17  House Style     

### Data Cleaning

In [4]:
#lower caps for column headers
test_data.columns = test_data.columns.str.lower()
#columns of dataset
test_data.columns[:]

Index(['id', 'pid', 'ms subclass', 'ms zoning', 'lot frontage', 'lot area',
       'street', 'alley', 'lot shape', 'land contour', 'utilities',
       'lot config', 'land slope', 'neighborhood', 'condition 1',
       'condition 2', 'bldg type', 'house style', 'overall qual',
       'overall cond', 'year built', 'year remod/add', 'roof style',
       'roof matl', 'exterior 1st', 'exterior 2nd', 'mas vnr type',
       'mas vnr area', 'exter qual', 'exter cond', 'foundation', 'bsmt qual',
       'bsmt cond', 'bsmt exposure', 'bsmtfin type 1', 'bsmtfin sf 1',
       'bsmtfin type 2', 'bsmtfin sf 2', 'bsmt unf sf', 'total bsmt sf',
       'heating', 'heating qc', 'central air', 'electrical', '1st flr sf',
       '2nd flr sf', 'low qual fin sf', 'gr liv area', 'bsmt full bath',
       'bsmt half bath', 'full bath', 'half bath', 'bedroom abvgr',
       'kitchen abvgr', 'kitchen qual', 'totrms abvgrd', 'functional',
       'fireplaces', 'fireplace qu', 'garage type', 'garage yr blt',
       'g

In [5]:
test_data.shape

(878, 80)

In [6]:
#convert 1 null value for total bsmt sf from NaN to 0 --> assume no basement since bsmt type is also null
test_data['total bsmt sf v2'] = test_data['total bsmt sf'].fillna(0)

#convert 1 null value for garage area to mean of all garage sizes --> property has a garage and the type is detached
mean=test_data['garage area'].mean()
test_data['garage area v2'] = test_data['garage area'].fillna(mean)

In [7]:
#to map all conditions related to proximity to transportation routes as '1'. 
#these are assumed to have negative impact on the price due to the noise
def transport_routes(x):
     if 'Artery' in x:
        return 1
     elif 'Feedr' in x:
        return 1
     elif 'RRAn' in x:
        return 1
     elif 'RRNn' in x:
        return 1
     elif 'RRAe' in x:
        return 1
     elif 'RRNe' in x:
        return 1
     else:
        return 0

test_data['transport1'] = test_data['condition 1'].map(transport_routes)

In [8]:
#create variable: age of property upon sale
test_data['property_age'] = test_data['yr sold']-test_data['year built']

In [9]:
#heating and fireplace quality of the house may be important predictor of property price as it is essential for the cold weather
#to create a new variable heat_quality to combine both features of 'heating qc' and 'fireplace qu'

#to rank 'fireplace qu' and 'heating qc'
test_data['fireplace qu v2'] = test_data['fireplace qu'].fillna('No')
test_data['fireplace_qu_scale'] = test_data['fireplace qu v2'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, 'No':0})
test_data['heating_qc_scale'] = test_data['heating qc'].map({'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1})

#new variable 'heat_quality' with min score of 1 and max score of 10.
test_data['heat_quality'] = test_data['fireplace_qu_scale'] + test_data['heating_qc_scale']

In [10]:
#to map for 'central air': "Y" = 1; "N" = 0
test_data['central_air_scale'] = test_data['central air'].map({'Y':1, 'N':0})

In [11]:
#one-hot encoding to create dummies for nominal variables: 'ms zoning', 'central_air_scale', 'transport1'.
test_data = pd.get_dummies(test_data, columns=['ms zoning','central_air_scale','transport1', 'neighborhood'])

In [12]:
test_data.columns[:]

Index(['id', 'pid', 'ms subclass', 'lot frontage', 'lot area', 'street',
       'alley', 'lot shape', 'land contour', 'utilities',
       ...
       'neighborhood_NoRidge', 'neighborhood_NridgHt', 'neighborhood_OldTown',
       'neighborhood_SWISU', 'neighborhood_Sawyer', 'neighborhood_SawyerW',
       'neighborhood_Somerst', 'neighborhood_StoneBr', 'neighborhood_Timber',
       'neighborhood_Veenker'],
      dtype='object', length=121)

In [13]:
#save csv file
test_data.to_csv('./datasets/test_cleaned.csv')